In [1]:
import pandas as pd
import pickle
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
# Connection string to connect to SQL Server named instance
conn_str = 'Driver=SQL Server;Server=.;Database=RevoScaleDb;Trusted_Connection=True;'
query ='SELECT * FROM  [dbo].[WineTrain] ORDER BY Id';
conn = pyodbc.connect(conn_str)
#Get traing dataset
wines = pd.read_sql(query,conn)

In [4]:
wines.head()

,Id,WineId,Facidity,Vacidity,Citric,Sugar,Chlorides,Fsulfur,Tsulfur,Density,pH,Sulphates,Alcohol,Quality,Color
0,1,1,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,2,2,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,3,3,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,4,4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,5,5,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


In [5]:
#Clean dataset
features= wines.columns.drop(["Id", "WineId","Color","Quality"])
columns = wines.columns.drop(["Id", "WineId","Color"])
target ="Quality"
x = wines[features]
y = wines[target]

In [5]:
correlation = wines[columns].corr()
correlation["Quality"].sort_values(ascending=False)

Quality      1.000000
Alcohol      0.459498
Citric       0.076710
Fsulfur      0.050007
pH           0.049022
Sulphates    0.042432
Tsulfur     -0.053392
Sugar       -0.056267
Facidity    -0.061785
Chlorides   -0.182914
Vacidity    -0.250617
Density     -0.307648
Name: Quality, dtype: float64

In [6]:
#Split dataset
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [7]:
x_train.head()

,Facidity,Vacidity,Citric,Sugar,Chlorides,Fsulfur,Tsulfur,Density,pH,Sulphates,Alcohol
1491,7.5,0.20,0.49,1.3,0.031,8.0,97.0,0.99180,3.06,0.62,11.1
3289,6.6,0.28,0.42,8.2,0.044,60.0,196.0,0.99562,3.14,0.48,9.4
2080,6.8,0.36,0.32,1.6,0.039,10.0,124.0,0.99480,3.30,0.67,9.6
3504,8.8,0.41,0.64,2.2,0.093,9.0,42.0,0.99860,3.54,0.66,10.5
4033,8.3,0.58,0.13,2.9,0.096,14.0,63.0,0.99840,3.17,0.62,9.1


In [7]:
# Initialize the model class.
lin_model = LinearRegression()
# Fit the model to the training data.
lin_model.fit(x_train,y_train)
#Returns the coefficient of determination R^2 of the prediction
accuracy = lin_model.score(x_test, y_test)
print("Accuracy: {}%".format(int(round(accuracy * 100))))

Accuracy: 29%


In [9]:
# Make predictions using the testing set
y_pred = lin_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
mse

0.55902077508312253

In [8]:
#Serialize model to file
modelFile ="d:\\Repos\\Cloud4YourData\\Demos\\4DevKatowice2018\\MLRevoscale\\Data\\Models\\lin_model.pkl"
with open(modelFile, 'wb') as file:
    pickle.dump(lin_model, file)

In [9]:
#Save model in Database
lin_model_ser = pickle.dumps(lin_model)
#Save model
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
cursor.execute("DELETE FROM dbo.Models WHERE ModelName ='LinReg' AND ModelLanguage ='Python'");
cursor.execute("INSERT INTO dbo.Models(ModelName,ModelLanguage,Model) values (?,?,?)", 
               'LinReg','Python',lin_model_ser)
conn.commit()

In [12]:
#Support Vector Regression (SVR)
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf')
svr_model.fit(x_train, y_train)
#Returns the coefficient of determination R^2 of the prediction
accuracy = svr_model.score(x_test, y_test)
print("Accuracy: {}%".format(int(round(accuracy * 100))))

Accuracy: 32%


In [13]:
y_pred = svr_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
mse

0.53415671095336414

In [14]:
#Serialize model to file
modelFile ="d:\\Repos\\Cloud4YourData\\Demos\\4DevKatowice2018\\MLRevoscale\\Data\\Models\\svr_model.pkl"
with open(modelFile, 'wb') as file:
    pickle.dump(svr_model, file)

In [15]:
#Save model in Database
svm_model_ser = pickle.dumps(svr_model)
#Save model
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
cursor.execute("DELETE FROM dbo.Models WHERE ModelName ='SVR' AND ModelLanguage ='Python'");
cursor.execute("INSERT INTO dbo.Models(ModelName,ModelLanguage,Model) values (?,?,?)", 
               'SVR','Python',svm_model_ser)
conn.commit()